# Query & Key & Value

Transformer 모델의 multi-head attention 메커니즘은 입력 데이터에 대해 다양한 위치의 정보를 고려하여 연산을 수행한다. <br>
이 과정에서 입력 데이터는 세 가지 요소인 Query (Q), Key (K), Value (V)로 변환된다.

`Query`<br>
현재 입력 데이터의 정보를 표현하며, 다른 입력과의 관계를 탐색하는 데 사용된다.

`Key`<br>
입력 데이터의 각 요소에 대한 "고유한 식별자"로 작용하며, Query와 비교하여 관련성을 평가하는 데 사용된다.

`Value`<br>
Query와 Key의 관계가 성립될 때 참조되는 실제 정보이며, attention score에 따라 가중치가 조정되어 최종 출력에 반영된다.


위와 같은 `Q`, `K`, `V`를 만드는 코드를 확인하겠다. 

In [1]:
import torch
import torch.nn as nn

Transformer 모델의 경우 입력과 출력의 크기가 항상 동일하다는 특징이 있다.

In [2]:
batch = 64
seq = 1024
n_dim = 768
n_head = 16

X = torch.randn(batch, seq, n_dim)

## `nn.Linear` + `torch.split` OR `torch.chunk`

`torch.split` 

- **split_size_or_sections**

: 각 부분의 크기(정수) 또는 각 부분의 크기를 나열한 리스트

- **dim**

: split 할 대상 차원을 선택한다.

In [10]:
linear = nn.Linear(n_dim, 3*n_dim, bias=False)
q, k, v = linear(X).split(n_dim, dim=-1) # 마지막 차원 (n_dim)을 대상으로 각 768개씩 쪼갠다.
print(q.shape, k.shape, v.shape)

torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768])


`torch.chunk`

- **chunks**

: 반환 할 부분(청크)의 수


- **dim**

: 구분 할 대상 차원을 선택한다.

In [11]:
q, k, v = linear(X).chunk(3, dim=-1)
print(q.shape, k.shape, v.shape)

torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768])


## Only `nn.Linear`

In [12]:
q_linear = nn.Linear(n_dim, n_dim, bias=False)
k_linear = nn.Linear(n_dim, n_dim, bias=False)
v_linear = nn.Linear(n_dim, n_dim, bias=False)

q = q_linear(X)
k = k_linear(X)
v = v_linear(X)

print(q.shape, k.shape, v.shape)

torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768]) torch.Size([64, 1024, 768])


# Multi Head를 위한 분리

Transformer 모델은 Multi-Head Attention을 수행하므로 Q, K, V 각각을 헤드수에 맞게 차원을 조정해야한다.<br>
이때 `view`와 `transpose`함수를 통해 조절된다.

Multi Head Attention에 들어갈 각 차원은 **`(배치, 헤드 수, 시퀀스, 차원//헤드 수)`** 이다.

In [13]:
k = k.view(batch, seq, n_head, n_dim//n_head).transpose(1, 2)
v = v.view(batch, seq, n_head, n_dim//n_head).transpose(1, 2)
q = q.view(batch, seq, n_head, n_dim//n_head).transpose(1, 2)

In [14]:
print(k.shape, v.shape, q.shape)

torch.Size([64, 16, 1024, 48]) torch.Size([64, 16, 1024, 48]) torch.Size([64, 16, 1024, 48])
